# Question 4: The West Coast Heatwave, Figure 1

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

from pathlib import Path

# Small style adjustments for more readable plots
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

In [2]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import fsspec

import xarray as xr
xr.set_options(display_style="html")  # display dataset nicely

import warnings
warnings.simplefilter("ignore")  # filter some warning messages

# code features from https://scitools.org.uk/cartopy/docs/v0.14/matplotlib/feature_interface.html
crs = ccrs.PlateCarree()  # set projection

In [3]:
ds = xr.open_dataset("~/shared/climate-data/ds_hw.nc")

## Calculating SST Data from 2002-2012 for Anomaly Calculations

Figure 1 shows the monthly anomalies for 2014-2016, using SST from 2002-2012 per month and per location as the baseline. So, we first need to calculate this baseline using a 30-day running average of SST, and then subtract the data from the baseline to get the anomaly calculations.

In [ ]:
trend_data = ds.analysed_sst.sel(time = slice('2002', '2012'))

trend = trend_data.rolling(time=30).mean(skipna=True) # get 30-day running average 
baseline = trend.groupby('time.month').mean(skipna=True) # group by day of year 

test = ds.analysed_sst.sel(time = slice('2014', '2016')) # get 2014-2016 data
anomalies = test.groupby('time.month') - baseline # calculate anomalies as observed data - baseline average

In [ ]:
anomalies

## Creating Figures

We now plot the anomalies for 2014, 2015, and 2016 (only through September for 2016) based on location.

In [ ]:
def annotate_map(ax, month):
    """
    Helper function for adding latitude/longitude ticks,
    land lines, and subplot letter label to map.
    """
    # add lat/lon axis ticks
    ax.set_yticks([35, 40, 45], crs=crs)
    ax.set_ylabel(None)
    ax.set_xticks([-130, -120], crs=crs)
    ax.set_xlabel(None)

    # add land & coastlines
    ax.coastlines("10m", color="k")
    ax.add_feature(cfeature.LAND, color="grey")
    ax.add_feature(cfeature.STATES.with_scale("10m"))
    
    # add subplot month label in the top right
    ax.text(-118.5, 48, month, c='white', size='xx-large',
            weight=600, ha='right', va='top')

In [ ]:
def make_fig(ax, month, year):
    
    # colorbar customization
    cbar_kwargs = {
        'location': 'bottom',
        'orientation': 'horizontal',
        'pad': 0.1,
        'aspect': 30,
        'label': '$\Delta$ SST ($\degree$C)',
        'ticks': np.arange(-2, 3, 2)
    }

    year.plot(ax=ax, transform=crs, cmap='jet',
                        cbar_kwargs=cbar_kwargs,
                        vmin=0, vmax=5
    )

    annotate_map(ax, letter = month)

fig = plt.figure(figsize=(5, 8))
ax = plt.subplot(projection=crs)
make_fig(ax)
#fig.savefig("outputs/Q04.png", bbox_inches="tight")